## Binder quick start (safe defaults)

- Optional: set `INAT_TOKEN` in this session for higher limits.
- Start with low-traffic settings: `fast=True`, `per_page=25`, `max_pages=2`, `fetch_images=False`.
- This notebook runs without a token (anonymous), but may be rate-limited.

In [2]:
import os
#from pynat.helpers import coming_soon, get_park_data
from helpers import coming_soon, get_park_data

INAT_TOKEN_SET = bool(os.getenv("INAT_TOKEN"))
print(f"INAT token set: {INAT_TOKEN_SET}")
print("Low-traffic defaults: fast=True, per_page=25, max_pages=2, fetch_images=False")

# Example usage (uncomment to run)
# df = coming_soon(
#     "birds",
#     loc=(37.6669, -77.8883, 25),
#     fast=True,
#     per_page=25,
#     max_pages=2,
#     fetch_images=False,
# )
# display(df.head(20))

INAT token set: False
Low-traffic defaults: fast=True, per_page=25, max_pages=2, fetch_images=False


In [3]:
import pandas as pd
import pyinaturalist as inat

In [4]:
# get geogenters for all parks

parks = {
    'tucker': (37.66713, -77.88739, 0.4), 
    'hiddenrock': (37.70219, -77.87333, 0.63), 
    'leakesmill': (37.70678, -77.95971, 0.68), 
    'matthews': (37.78714, -78.03979, 0.34),
}

In [5]:
res = {}
for park in parks:
    pagenum = 0
    res[park] = pd.DataFrame()
    while(True):
        pagenum += 1
        tmp = pd.json_normalize(inat.get_observation_species_counts(lat=parks[park][0], 
                                                                    lng=parks[park][1], 
                                                                    radius=parks[park][2], 
                                                                    verifiable=True,
                                                                    #capitve
                                                                    #introduced
                                                                    #native
                                                                    #endemic
                                                                    #threatened
                                                                    page=pagenum,
                                                                   )['results'])
        if tmp.empty:
            break
        res[park] = pd.concat([res[park], tmp])    
    normer = pd.json_normalize(inat.get_observation_species_counts(taxon_id=res[park]['taxon.id'].to_list(),
                                                                   place_id=1297, # relative to Virginia
                                                                   verifiable=True,
                                                                  )['results'])
    res[park]['normalizer'] = res[park]['taxon.id'].map(normer.set_index('taxon.id')['count'])
    res[park]['sorter'] = res[park]['count']/res[park]['normalizer']
    res[park].sort_values('sorter', ascending=False, inplace=True)

### suspicious that so many of these max at 500...

In [6]:
for park in parks:
    print(f"{park}: {res[park].shape}")

tucker: (591, 45)
hiddenrock: (218, 36)
leakesmill: (269, 45)
matthews: (72, 36)


In [7]:
res['tucker'][['count', 'taxon.name', 'taxon.preferred_common_name', 'taxon.wikipedia_url']].sort_values('count', ascending=False).head(25)

,count,taxon.name,taxon.preferred_common_name,taxon.wikipedia_url
0,34,Sialia sialis,Eastern Bluebird,http://en.wikipedia.org/wiki/Eastern_bluebird
1,31,Argia apicalis,Blue-fronted Dancer,http://en.wikipedia.org/wiki/Blue-fronted_dancer
3,30,Hetaerina titia,Smoky Rubyspot,http://en.wikipedia.org/wiki/Hetaerina_titia
4,30,Ligustrum sinense,Chinese privet,http://en.wikipedia.org/wiki/Ligustrum_sinense
2,30,Argia moesta,Powdered Dancer,http://en.wikipedia.org/wiki/Powdered_dancer
5,24,Rosa multiflora,multiflora rose,http://en.wikipedia.org/wiki/Rosa_multiflora
7,19,Cyprinella analostana,Satinfin Shiner,http://en.wikipedia.org/wiki/Satinfin_shiner
6,19,Eupatorium serotinum,late boneset,http://en.wikipedia.org/wiki/Eupatorium_serotinum
8,19,Humulus scandens,Japanese Hops,http://en.wikipedia.org/wiki/Humulus_japonicus
9,17,Verbesina alternifolia,Wingstem,http://en.wikipedia.org/wiki/Verbesina_alterni...


In [45]:
res['leakesmill'][['count', 'taxon.name', 'taxon.preferred_common_name', 'taxon.wikipedia_url']].sort_values('count', ascending=False).head(25)

,count,taxon.name,taxon.preferred_common_name,taxon.wikipedia_url
0,18,Hepatica americana,round-lobed hepatica,http://en.wikipedia.org/wiki/Hepatica_nobilis
1,12,Erythronium umbilicatum,dimpled trout lily,http://en.wikipedia.org/wiki/Erythronium_umbil...
2,11,Tipularia discolor,crane-fly orchid,https://en.wikipedia.org/wiki/Tipularia_discolor
6,10,Stellaria pubera,star chickweed,http://en.wikipedia.org/wiki/Stellaria_pubera
3,10,Packera aurea,golden ragwort,http://en.wikipedia.org/wiki/Packera_aurea
4,10,Claytonia virginica,Virginia Springbeauty,https://en.wikipedia.org/wiki/Claytonia_virginica
5,10,Thalictrum thalictroides,rue anemone,http://en.wikipedia.org/wiki/Thalictrum_thalic...
7,9,Diphasiastrum digitatum,fan clubmoss,http://en.wikipedia.org/wiki/Diphasiastrum_dig...
8,9,Aplectrum hyemale,putty root,https://en.wikipedia.org/wiki/Aplectrum
10,6,Asarum minus,little heartleaf,https://en.wikipedia.org/wiki/Asarum_minus


In [46]:
res['hiddenrock'][['count', 'taxon.name', 'taxon.preferred_common_name', 'taxon.wikipedia_url']].sort_values('count', ascending=False).head(25)

,count,taxon.name,taxon.preferred_common_name,taxon.wikipedia_url
0,11,Lobelia inflata,Indian tobacco,https://en.wikipedia.org/wiki/Lobelia_inflata
1,9,Diphasiastrum digitatum,fan clubmoss,http://en.wikipedia.org/wiki/Diphasiastrum_dig...
2,8,Arisaema triphyllum,Jack-in-the-Pulpit,http://en.wikipedia.org/wiki/Arisaema_triphyllum
3,8,Lindera benzoin,northern spicebush,http://en.wikipedia.org/wiki/Lindera_benzoin
4,7,Podophyllum peltatum,mayapple,https://en.wikipedia.org/wiki/Podophyllum
5,6,Elaeagnus umbellata,autumn olive,http://en.wikipedia.org/wiki/Elaeagnus_umbellata
11,5,Microstegium vimineum,Japanese stiltgrass,http://en.wikipedia.org/wiki/Microstegium_vimi...
6,5,Monotropa uniflora,Ghost Pipe,http://en.wikipedia.org/wiki/Monotropa_uniflora
10,5,Packera aurea,golden ragwort,http://en.wikipedia.org/wiki/Packera_aurea
8,5,Saururus cernuus,lizard's tail,http://en.wikipedia.org/wiki/Saururus_cernuus


In [47]:
res['matthews'][['count', 'taxon.name', 'taxon.preferred_common_name', 'taxon.wikipedia_url']].sort_values('count', ascending=False).head(25)

,count,taxon.name,taxon.preferred_common_name,taxon.wikipedia_url
4,2,Rubus pensilvanicus,Pennsylvania Blackberry,http://en.wikipedia.org/wiki/Rubus_pensilvanicus
2,2,Erynnis juvenalis,Juvenal's Duskywing,http://en.wikipedia.org/wiki/Erynnis_juvenalis
0,2,Acer rubrum,red maple,http://en.wikipedia.org/wiki/Acer_rubrum
1,2,Prunus serotina,black cherry,https://en.wikipedia.org/wiki/Prunus_serotina
5,2,Diastrophus nebulosus,Blackberry Knot Gall Wasp,https://en.wikipedia.org/wiki/Diastrophus_nebu...
3,2,Lonicera japonica,Japanese honeysuckle,https://en.wikipedia.org/wiki/Lonicera_japonica
9,1,Plethodon cinereus,Eastern Red-backed Salamander,http://en.wikipedia.org/wiki/Red-backed_salama...
39,1,Elaeagnus umbellata,autumn olive,http://en.wikipedia.org/wiki/Elaeagnus_umbellata
45,1,Viola sororia,common blue violet,http://en.wikipedia.org/wiki/Viola_sororia
25,1,Trametes versicolor,turkey-tail,https://en.wikipedia.org/wiki/Trametes_versicolor


In [ ]:
# highlight unusual species esp. engagered ones (sum and check trends)

In [ ]:
# highlight unusual or *unusually-increasing* invasives (sum and check trends)